In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics.pairwise import euclidean_distances
from itertools import combinations

In [2]:
# Set the random seed for the coursework
np.random.seed(124)

In [3]:
# Define the polynomial kernel
def polynomial_kernel(p, q, d):
    return (p @ q.T) ** d

# Define the gaussian kernel
def gaussian_kernel(p, q, c):
    return np.exp(-c * euclidean_distances(p, q) ** 2)

# Get the list of combinations given the list of classes
def get_combinations(classes):
    return list(combinations(classes, 2))

In [4]:
# Retrieve the data
labels = []
images = []

with open('./dataset/zipcombo.dat', 'r') as file:
    for line in file:
        values = list(map(float, line.strip().split()))
        labels.append(values[0])
        images.append(np.array(values[1:]))
    
labels = np.array(labels)
images = np.array(images)

In [5]:
class OvOKernelPerceptron:
    def __init__(self, epochs, X_train, X_test, y_train, y_test, kernel_type, kernel_param):
        self.epochs = epochs  # Number of times the training data will be iterated over
        self.X_train = X_train 
        self.X_test = X_test  
        self.y_train = y_train  
        self.y_test = y_test  
        
        # Calculate the number of unique classes in the dataset
        self.num_classes = len(np.unique(np.append(y_train, y_test)))
        self.train_size = X_train.shape[0]  # Number of training examples
        self.test_size = X_test.shape[0]  # Number of testing examples
        self.classifiers = get_combinations(np.arange(0, 10)) # Get all combinations of classifiers for OvO
        self.num_classifiers = len(self.classifiers) # # Number of classifiers needed for OvO 
        
        # Initialise the alpha coefficients for each classifier and training example
        self.alpha = np.zeros((self.num_classifiers, self.train_size))

        # Initialise the kernel based on the specified type and parameter
        if kernel_type == "polynomial":
            self.K_train = polynomial_kernel(self.X_train, self.X_train, kernel_param)
            self.K_test = polynomial_kernel(self.X_test, self.X_train, kernel_param)
        elif kernel_type == "gaussian":
            self.K_train = gaussian_kernel(self.X_train, self.X_train, kernel_param)
            self.K_test = gaussian_kernel(self.X_test, self.X_train, kernel_param)
        
    def predict(self, K):
        # Make a prediction for the i-th example
        votes = self.alpha @ K # Get the votes for all classifiers
        class_votes = np.zeros(self.num_classes) # Initialise the array to record the vote for each class
        
        for j, vote in enumerate(votes):
            combo = self.classifiers[j]
            
            # If vote > 0, then vote for the first class in the combination, else vote for the second class
            voted_class = combo[0] if vote > 0 else combo[1] 
            class_votes[voted_class] += 1
        
        # Return the class with the most votes
        return np.argmax(class_votes) 

    def fit(self):
        # Train the perceptron model
        errors = 0
        for epoch in range(self.epochs):
            for t in range(self.train_size):
                K_t = self.K_train[t]
                y_t = int(self.y_train[t])
                y_pred = self.predict(K_t)
                if y_pred != y_t:
                    # Update alpha coefficients for misclassified examples
                    for j, combo in enumerate(self.classifiers):
                        if y_t in combo:
                            update = 1 if combo[0] == y_t else -1
                            self.alpha[j, t] += update
                    errors += 1
        # Return the percentage error over all epochs
        return errors / (self.epochs * self.train_size) * 100

    def test(self):
        # Evaluate the model on the test dataset
        errors = 0
        for i in range(self.test_size):
            K_i = self.K_test[i]
            y_i = int(self.y_test[i])
            y_pred = self.predict(K_i)
            if y_i != y_pred:
                errors += 1
                
        return errors / self.test_size * 100


In [6]:
def train_and_test_model(X_train, X_test, y_train, y_test, kernel_type, kernel_param, return_model=False):
    # Initialize the perceptron model with specified kernel and kernel parameter
    ovo_kernel_perceptron = OvOKernelPerceptron(5, X_train, X_test, y_train, y_test, kernel_type, kernel_param)
    
    # Train model and record training error rate
    train_error_rate = ovo_kernel_perceptron.fit()
    
    # Test model and record testing error rate
    test_error_rate = ovo_kernel_perceptron.test()
    
    if return_model:
        return ovo_kernel_perceptron, test_error_rate
    else:
        return train_error_rate, test_error_rate

def cross_validate_for_best_parameter(X_train, y_train, kernel_type, param_list, n_splits=5):
    # Initialize KFold for cross-validation
    kf = KFold(n_splits=n_splits)
    val_error_rates = np.zeros(len(param_list))

    # Loop over each fold in cross-validation
    for train_index, val_index in kf.split(X_train):
        # Create training and validation sets for this fold
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        
        # Evaluate each parameter using cross-validation
        for i, param in enumerate(param_list):
            train_error, val_error = train_and_test_model(X_train_fold, X_val_fold, y_train_fold, y_val_fold, kernel_type, param)
            val_error_rates[i] += val_error

    # Determine the best parameter with minimum validation error rate
    best_param = param_list[np.argmin(val_error_rates)]
    return best_param

In [7]:
# Part 3 Q6.1

# Initialise parameters
degree_list = np.arange(1, 8)
num_runs = 20
num_degrees = len(degree_list)

# Initialise arrays to store error rates and stds
train_error_rates, test_error_rates = np.zeros(num_degrees), np.zeros(num_degrees)
train_stds, test_stds = np.zeros(num_degrees), np.zeros(num_degrees)

for degree in degree_list:
    degree_train_errors, degree_test_errors = np.zeros(num_runs), np.zeros(num_runs)
    
    for run in tqdm(range(num_runs)):
        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)
        
        train_error_rate, test_error_rate = train_and_test_model(X_train, X_test, y_train, y_test, 'polynomial', degree)
        degree_train_errors[run], degree_test_errors[run] = train_error_rate, test_error_rate

    # Print results for current degree
    print(f"Degree: {degree} \n    Mean train error rate: {degree_train_errors.mean():.3f}%±\
{degree_train_errors.std():.3f}%, Mean test error rate: \
{degree_test_errors.mean():.3f}%±{degree_test_errors.std():.3f}%")        
        
    # Store the mean and std of errors for each degree
    train_error_rates[degree - 1] = degree_train_errors.mean()
    test_error_rates[degree - 1] = degree_test_errors.mean()
    train_stds[degree - 1] = degree_train_errors.std()
    test_stds[degree - 1] = degree_test_errors.std()

100%|███████████████████████████████████████████████████████████████████| 20/20 [00:52<00:00,  2.64s/it]


Degree: 1 
    Mean train error rate: 10.589%±0.237%, Mean test error rate: 9.941%±0.618%


100%|███████████████████████████████████████████████████████████████████| 20/20 [01:00<00:00,  3.02s/it]


Degree: 2 
    Mean train error rate: 6.505%±0.222%, Mean test error rate: 6.825%±0.603%


100%|███████████████████████████████████████████████████████████████████| 20/20 [01:08<00:00,  3.41s/it]


...

Degree: 5 
    Mean train error rate: 2.605%±0.145%, Mean test error rate: 4.960%±0.525%


100%|███████████████████████████████████████████████████████████████████| 20/20 [00:57<00:00,  2.87s/it]


Degree: 6 
    Mean train error rate: 2.336%±0.113%, Mean test error rate: 4.772%±0.443%


100%|███████████████████████████████████████████████████████████████████| 20/20 [01:10<00:00,  3.54s/it]

Degree: 7 
    Mean train error rate: 2.190%±0.105%, Mean test error rate: 4.718%±0.495%


In [8]:
# Part3 Q6.2

# Initialise arrays for storing results
num_runs = 20
best_degrees = np.zeros(num_runs)
test_error_rates = np.zeros(num_runs)


for run in tqdm(range(num_runs)):
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

    # Determine and record the best degree of the run
    best_degree = cross_validate_for_best_parameter(X_train, y_train, 'polynomial', degree_list)
    best_degrees[run] = best_degree

    # Retrain on full 80% training data with best degree and test
    train_error_rate, test_error_rate = train_and_test_model(X_train, X_test, y_train, y_test, 'polynomial', best_degree)
    test_error_rates[run] = test_error_rate
    

# Calculate and print the mean and std of test error and best degree
mean_test_error = test_error_rates.mean()
std_test_error = test_error_rates.std()
mean_best_degree = best_degrees.mean()
std_best_degree = best_degrees.std()

print(f"Mean test error: {mean_test_error:.3f}±{std_test_error:.3f}")
print(f"Mean best degree: {mean_best_degree:.3f}±{std_best_degree:.3f}\n")

for i in range(num_runs):
    print(f"run: {i}, d*={best_degrees[i]}, test error rate={test_error_rates[i]:.3f}")

100%|███████████████████████████████████████████████████████████████████| 20/20 [22:30<00:00, 67.53s/it]

Mean test error: 4.761±0.643
Mean best degree: 6.550±0.589

run: 0, d*=5.0, test error rate=5.269
run: 1, d*=7.0, test error rate=4.516
run: 2, d*=6.0, test error rate=4.462
run: 3, d*=6.0, test error rate=4.194
run: 4, d*=6.0, test error rate=4.462
run: 5, d*=6.0, test error rate=4.785
run: 6, d*=7.0, test error rate=5.484
run: 7, d*=7.0, test error rate=4.301
run: 8, d*=7.0, test error rate=3.871
run: 9, d*=6.0, test error rate=6.129
run: 10, d*=7.0, test error rate=5.430
run: 11, d*=7.0, test error rate=4.355
run: 12, d*=7.0, test error rate=6.129
run: 13, d*=7.0, test error rate=4.839
run: 14, d*=7.0, test error rate=4.892
run: 15, d*=7.0, test error rate=4.624
run: 16, d*=6.0, test error rate=4.409
run: 17, d*=7.0, test error rate=3.710
run: 18, d*=7.0, test error rate=5.054
run: 19, d*=6.0, test error rate=4.301
